In [1]:
import pandas as pd
import pandas_dedupe

In [2]:
from pandas_dedupe import dedupe_dataframe

In [3]:
import re

In [4]:
outlets_raw = pd.read_excel('outlets_raw.xlsx', encoding='utf-8')

In [5]:
# удаляем часть названия точки по шаблону 'все, что после ", '

outlets_raw['Торг_точка_грязная'] = outlets_raw['Торг_точка_грязная'].str.replace('",.*', '')

In [6]:
# удаляем часть названия точки по шаблону 'все, что не цифра и не буква на 1 месте '
   
outlets_raw['Торг_точка_грязная'] = outlets_raw['Торг_точка_грязная'].str.replace('^\\W', '')

In [7]:
# удаляем часть названия точки  ИП, ООО
   
outlets_raw['Торг_точка_грязная'] = outlets_raw['Торг_точка_грязная'].str.replace('(ИП|ООО)', '')

In [8]:
# добавляем пробелы после ,
   
outlets_raw['Торг_точка_грязная'] = outlets_raw['Торг_точка_грязная'].str.replace(',', ', ')

In [9]:
# добавляем пробелы после .
   
outlets_raw['Торг_точка_грязная'] = outlets_raw['Торг_точка_грязная'].str.replace('.', '. ')

In [10]:
# меняем - на пробелы
   
outlets_raw['Торг_точка_грязная'] = outlets_raw['Торг_точка_грязная'].str.replace('-', ' ')

In [11]:
# удаляем из названия все знаки пунктуации
   
outlets_raw['Торг_точка_грязная'] = outlets_raw['Торг_точка_грязная'].str.replace('[!"\#\$%\&\'\(\)\*\+,/:;<=>\-\.\?@_\[\]\^\{\|\}\~\\\\]', '')

In [12]:
# делаем все буквы в названии строчными

outlets_raw['Торг_точка_грязная'] = outlets_raw['Торг_точка_грязная'].str.lower()

In [13]:
# удаляем из названия все повторяющиеся пробелы
   
outlets_raw['Торг_точка_грязная'] = outlets_raw['Торг_точка_грязная'].str.replace(' +', ' ')

In [14]:
# удаляем из названия пробел, если он стоит в самом начале
   
outlets_raw['Торг_точка_грязная'] = outlets_raw['Торг_точка_грязная'].str.replace('^ ', '')

In [15]:
# удаляем часть адреса по шаблону 'все, что от ООО до вторых " '

outlets_raw['Торг_точка_грязная_адрес'] = outlets_raw['Торг_точка_грязная_адрес'].str.replace('^ООО.*".*"', '')

# удаляем часть адреса по шаблону 'все, что от ООО " '

outlets_raw['Торг_точка_грязная_адрес'] = outlets_raw['Торг_точка_грязная_адрес'].str.replace('^ООО.*".*', '')

# добавляем пробелы после ,
   
outlets_raw['Торг_точка_грязная_адрес'] = outlets_raw['Торг_точка_грязная_адрес'].str.replace(',', ', ')

# добавляем пробелы после .
   
outlets_raw['Торг_точка_грязная_адрес'] = outlets_raw['Торг_точка_грязная_адрес'].str.replace('.', '. ')

# меняем - на пробелы
   
outlets_raw['Торг_точка_грязная_адрес'] = outlets_raw['Торг_точка_грязная_адрес'].str.replace('-', ' ')

# удаляем из адреса все знаки пунктуации
   
outlets_raw['Торг_точка_грязная_адрес'] = outlets_raw['Торг_точка_грязная_адрес'].str.replace('[!"\#\$%\&\'\(\)\*\+,/:;<=>\-\.\?@_\[\]\^\{\|\}\~\\\\]', '')

# делаем все буквы в адресе строчными

outlets_raw['Торг_точка_грязная_адрес'] = outlets_raw['Торг_точка_грязная_адрес'].str.lower()

# удаляем из адреса все повторяющиеся пробелы
   
outlets_raw['Торг_точка_грязная_адрес'] = outlets_raw['Торг_точка_грязная_адрес'].str.replace(' +', ' ')

In [16]:
# удаляем адреса из 1 слова

outlets_raw['Торг_точка_грязная_адрес'] = outlets_raw['Торг_точка_грязная_адрес'].str.replace('^\\w*$', '')

In [17]:
# удаляем часть названия адреса по шаблону 'все, что не цифра и не буква на первом месте '

outlets_raw['Торг_точка_грязная_адрес'] = outlets_raw['Торг_точка_грязная_адрес'].str.replace('^\\W', '')

In [18]:
# создадим словарь {название точки: адрес точки} для точек из таблицы outlets_raw

d_adress = {key:[] for key in set(outlets_raw['Торг_точка_грязная'])}

for row in outlets_raw.itertuples():
    d_adress[row.Торг_точка_грязная] += [row.Торг_точка_грязная_адрес]
    d_adress[row.Торг_точка_грязная] = list(set(d_adress[row.Торг_точка_грязная]))  # оставляем уникальные значения адреса

In [19]:
# создадим еще 2 словаря: для замены "" и "он же" на настоящий адрес

d_null_adress = {}
d_same_adress = {}

for outlet, adresses in d_adress.items():
    if (len(adresses) == 2) and ('' in adresses) and ('он же' not in adresses):
        adresses.remove('')
        d_null_adress[outlet] = adresses[0]
    if (len(adresses) == 2) and ('' not in adresses) and ('он же' in adresses):
        adresses.remove('он же')
        d_same_adress[outlet] = adresses[0]
    if (len(adresses) == 3) and ('' in adresses) and ('он же' in adresses):
        adresses.remove('')
        adresses.remove('он же')
        d_null_adress[outlet] = adresses[0]
        d_same_adress[outlet] = adresses[0]

print(len(d_null_adress))
print(len(d_same_adress))

2525
116


In [20]:
# сделаем из словарей наборы данных

df_same_adress = pd.DataFrame.from_dict(d_same_adress, orient='index', columns=['Торг_точка_грязная_адрес'])
df_same_adress = df_same_adress.assign(Торг_точка_грязная=df_same_adress.index.values)

df_null_adress = pd.DataFrame.from_dict(d_null_adress, orient='index', columns=['Торг_точка_грязная_адрес'])
df_null_adress = df_null_adress.assign(Торг_точка_грязная=df_null_adress.index.values)

In [21]:
# заменим значения адресов для тех точек, где адрес указан как 'он же', при помощи объединения таблиц

outlets_with_adress = pd.merge(df_same_adress, outlets_raw, how='right', 
                               left_on='Торг_точка_грязная', right_on='Торг_точка_грязная')

In [22]:
# заменим значения адресов для тех точек, где адрес указан как 'он же', при помощи объединения таблиц

outlets_with_adress_same = pd.merge(df_same_adress, 
                               outlets_raw[outlets_raw['Торг_точка_грязная_адрес'] == 'он же'].drop('Торг_точка_грязная_адрес', 
                                                                                                    axis=1), 
                               how='right', 
                               left_on='Торг_точка_грязная', 
                               right_on='Торг_точка_грязная')

In [23]:
outlets_with_adress_same = outlets_with_adress_same.dropna()

In [24]:
# заменим значения адресов для тех точек, где адрес указан как '', при помощи объединения таблиц

outlets_with_adress_null = pd.merge(df_null_adress, 
                               outlets_raw[outlets_raw['Торг_точка_грязная_адрес'] == ''].drop('Торг_точка_грязная_адрес', 
                                                                                                    axis=1), 
                               how='right', 
                               left_on='Торг_точка_грязная', 
                               right_on='Торг_точка_грязная')

In [25]:
outlets_with_adress_null = outlets_with_adress_null.dropna()

In [26]:
# собираем предыдущие шаги в общую таблицу для дедубликации

outlets_to_dedupe = pd.merge(outlets_raw, pd.concat([outlets_with_adress_same, outlets_with_adress_null], ignore_index=True).drop(['outlet_clean_id', 'Город дистрибьютора', 'Торг_точка_грязная'], axis=1), 
                             how='left', left_on='id', right_on='id')

In [27]:
# убираем nan

outlets_to_dedupe.fillna('', inplace=True)

In [28]:
# заменяем 'он же' на адрес

outlets_to_dedupe.Торг_точка_грязная_адрес_x.replace('он же', outlets_to_dedupe.Торг_точка_грязная_адрес_y, inplace=True)

In [29]:
# заменяем '' на адрес

outlets_to_dedupe.Торг_точка_грязная_адрес_x.replace('', outlets_to_dedupe.Торг_точка_грязная_адрес_y, inplace=True)
outlets_to_dedupe = outlets_to_dedupe.drop('Торг_точка_грязная_адрес_y', axis=1)

In [30]:
# меняем название столбца

outlets_to_dedupe.rename(columns={'Торг_точка_грязная_адрес_x': 'Торг_точка_грязная_адрес'}, inplace=True)

In [31]:
# добавим числа вместо адреса со значением ''

outlets_to_dedupe.Торг_точка_грязная_адрес.replace('', 1, inplace=True)

In [32]:
# производим дедубликацию

dedupe_outlets = dedupe_dataframe(outlets_to_dedupe, ['Торг_точка_грязная', 'Торг_точка_грязная_адрес'])

importing data ...
reading from dedupe_dataframe_learned_settings


clustering...
# duplicate sets 4575


In [33]:
# оставляем только значения с confidence > 0.4

dedupe_outlets = dedupe_outlets[dedupe_outlets['confidence'] > 0.4]

In [34]:
# загрузим таблицу с исходными данными еще раз

outlets_raw_to_join = pd.read_excel('outlets_raw.xlsx', encoding='utf-8')

In [35]:
# изменим тип данных в столбце 'id'

outlets_raw_to_join['id'] = outlets_raw_to_join['id'].astype(str)

In [36]:
# объединим таблицы 

output = pd.merge(outlets_raw_to_join.drop('outlet_clean_id', axis=1), dedupe_outlets[['id', 'cluster id']], how='left', left_on='id', right_on='id')

In [37]:
# убираем nan и преобразовываем значения в целые числа

output.fillna(0, inplace=True)
output['cluster id'] = output['cluster id'].astype(int)

In [38]:
# переименовываем столбец "cluster id" в "outlet_clean_id"

output.rename(columns={'cluster id': 'outlet_clean_id'}, inplace=True)

In [39]:
# сохраняем таблицу в Excel

output.to_csv("output.csv", index=False)

In [40]:
# создаем таблицу "outlets_clean"

outlets_clean = output[['Торг_точка_грязная_адрес', 'outlet_clean_id', 'id']]

In [41]:
# находим максимальный id 

max_id = max(outlets_clean.outlet_clean_id)

In [42]:
# создадим маску и разобьем outlets_clean

mask = outlets_clean['outlet_clean_id'] == 0

outlets_clean_without_id, outlets_clean_with_id = outlets_clean[mask], outlets_clean[~mask]

In [43]:
# изменим тип данных столбца id

outlets_clean_without_id['id'] = outlets_clean_without_id['id'].astype(int)

c:\users\user\miniconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [44]:
# создадим столбец со значениями outlet_clean_id

outlet_clean_id = pd.DataFrame({'outlet_clean_id':range(max_id + 1, max_id+len(outlets_clean_without_id) + 1),
                               'id':outlets_clean_without_id.index.values})
outlet_clean_id.set_index(outlets_clean_without_id.index.values)

,outlet_clean_id,id
0,4575,0
4,4576,4
7,4577,7
9,4578,9
13,4579,13
...,...,...
20178,12770,20178
20183,12771,20183
20197,12772,20197
20202,12773,20202


In [45]:
# обновим значения outlet_clean_id в таблице без outlet_clean_id

outlets_clean_without_id = pd.merge(outlet_clean_id, outlets_clean_without_id.drop('outlet_clean_id', axis=1), 
                                    how='left', left_on='id', right_on='id').drop('id', axis=1)

In [46]:
# соберем таблицу outlets_clean

outlets_clean = pd.concat([outlets_clean_without_id, outlets_clean_with_id], ignore_index=True, sort=False)

In [47]:
outlets_clean = outlets_clean.drop('id', axis=1).fillna('-')

In [48]:
# переименуем столбцы в таблице outlets_clean

outlets_clean.rename(columns={'Торг_точка_грязная_адрес': 'Торг_точка_чистый_адрес', 'outlet_clean_id': 'id'}, inplace=True)

In [49]:
# отсортируем таблицу и удалим дубликаты id

outlets_clean = outlets_clean.sort_values(by=['id', 'Торг_точка_чистый_адрес'])
outlets_clean = outlets_clean.drop_duplicates(subset='id', keep='last')

In [50]:
# сохраняем таблицу в Excel

outlets_clean.to_csv("outlets_clean.csv", index=False)